In [6]:
%pip install gradio opencv-python torch torchvision numpy matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import cv2
import torch
import torch.nn as nn
import numpy as np
import gradio as gr
from torchvision import transforms


In [8]:
import torch
import torch.nn as nn

class CSRNet(nn.Module):
    def __init__(self):
        super(CSRNet, self).__init__()

        self.frontend = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(256, 512, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(inplace=True)
        )

        self.backend = nn.Sequential(
            nn.Conv2d(512, 512, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 256, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(128, 64, 3, dilation=2, padding=2), nn.ReLU(inplace=True)
        )

        # Must match training exactly
        self.output = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        x = self.frontend(x)
        x = self.backend(x)
        x = self.output(x)
        return x


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CSRNet().to(device)
model.load_state_dict(torch.load("csrnet_finalb.pth", map_location=device))
model.eval()


CSRNet(
  (frontend): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilatio

In [10]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [11]:
def process_frame(frame, threshold=25, show_heatmap=False):
    # Resize for Part B
    frame_resized = cv2.resize(frame, (512, 512))
    img_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)

    input_tensor = transform(img_rgb).unsqueeze(0).to(device)

    with torch.no_grad():
        density_map = model(input_tensor)

    count = density_map.sum().item()

    alert = "NORMAL"
    if count > threshold:
        alert = "⚠️ CROWD LIMIT EXCEEDED"

    # Optional heatmap
    if show_heatmap:
        dm = density_map.squeeze().cpu().numpy()
        dm = cv2.normalize(dm, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        heatmap = cv2.applyColorMap(dm, cv2.COLORMAP_JET)
        heatmap = cv2.resize(heatmap, frame.shape[1::-1])
        overlay = cv2.addWeighted(frame, 0.6, heatmap, 0.4, 0)
    else:
        overlay = frame

    return overlay, f"Count: {int(count)}", alert


In [12]:
def webcam_stream(threshold, show_heatmap):
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        output, count_text, alert = process_frame(
            frame, threshold, show_heatmap
        )

        yield output, count_text, alert

    cap.release()


In [13]:
interface = gr.Interface(
    fn=webcam_stream,
    inputs=[
        gr.Slider(1, 100, value=25, label="Crowd Threshold"),
        gr.Checkbox(label="Show Density Heatmap")
    ],
    outputs=[
        gr.Image(type="numpy", label="Live Feed"),
        gr.Textbox(label="Crowd Count"),
        gr.Textbox(label="Alert Status")
    ],
    live=True
)


In [14]:
interface.launch()

* Running on local URL:  http://127.0.0.1:7860


ValueError: When localhost is not accessible, a shareable link must be created. Please set share=True or check your proxy settings to allow access to localhost.